# <font color="#1b5776">1. <u>Adding Imports</u></font>

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
from collections import OrderedDict
import pickle

# <font color="#1b5776">2. <u>Getting the Data</u></font>

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/train_featureEngineered1.csv",dtype={"fullVisitorId":"string"})

test = pd.read_csv("/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/test_featureEngineered1.csv",dtype={"fullVisitorId":"string"})

submit = pd.read_csv("/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/sample_submission_v2.csv.zip",dtype={"fullVisitorId":"string"})

<font color="#8fce00"><h4><u><b>Note:</b></u></h4></font> We have already utilized the dayName feature for creating different new features. We can just drop it now.

In [ ]:
del train["dayName"]
del test["dayName"]

In [ ]:
train.head(2)

,fullVisitorId,channelGrouping,date,visitNumber,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_networkDomain,totals_hits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_transactions,totals_transactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_referralPath,trafficSource_adContent,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.adNetworkType,value,year,month,dayOfMonth,dayOfWeek,weekOfYear,dayOfYear,quarter,isYearEnd,is_weekend,visit_hr,visit_min,visit_sec,time_of_day,is_office_hours
0,3162355547410993243,Organic Search,2017-10-16,1,Firefox,Windows,No,desktop,Europe,Western Europe,Germany,OTHERS,OTHERS,OTHERS,OTHERS,1,1.0,1.0,0.0,0.0,0.0,OTHERS,google,organic,water bottle,OTHERS,OTHERS,0.0,OTHERS,EMEA,2017,10,16,0,42,289,4,Yes,No,0,0,50,midnight(12am-6am),No
1,8934116514970143966,Referral,2017-10-16,6,Chrome,Chrome OS,No,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Cupertino,OTHERS,2,2.0,2.0,28.0,0.0,0.0,OTHERS,sites.google.com,referral,OTHERS,/a/google.com/transportation/mtv-services/bike...,OTHERS,0.0,OTHERS,North America,2017,10,16,0,42,289,4,Yes,No,17,51,47,afternoon_evening(12pm-6pm),Yes


In [ ]:
test.head(2)

,fullVisitorId,channelGrouping,date,visitNumber,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_networkDomain,totals_hits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_transactions,totals_transactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_referralPath,trafficSource_adContent,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.adNetworkType,value,year,month,dayOfMonth,dayOfWeek,weekOfYear,dayOfYear,quarter,isYearEnd,is_weekend,visit_hr,visit_min,visit_sec,time_of_day,is_office_hours
0,7460955084541987166,Organic Search,2018-05-11,2,Chrome,Android,Yes,mobile,Asia,Southern Asia,India,Delhi,OTHERS,OTHERS,OTHERS,4,3.0,1,973.0,0.0,0.0,OTHERS,google,organic,OTHERS,OTHERS,OTHERS,0.0,OTHERS,APAC,2018,5,11,4,19,131,2,No,No,4,29,1,midnight(12am-6am),No
1,460252456180441002,Direct,2018-05-11,166,Chrome,Macintosh,No,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,San Francisco,OTHERS,4,3.0,1,49.0,0.0,0.0,OTHERS,(direct),OTHERS,OTHERS,OTHERS,OTHERS,0.0,OTHERS,North America,2018,5,11,4,19,131,2,No,No,18,48,3,night(6pm-12am),No


<font color="#8fce00"><h4><u><b>Note:</b></u></h4></font> 
We have 3 years in our dataset: 2016,2017 & 2018. And we have different revenues generated in different year (also: data for different months is given in these years). Hence, instead of treating them as integer, let's convert the <font color="#da70d6"><b>year</b></font> feature into categorical feature.

In [ ]:
train['year'] = train['year'].astype("object")
test['year'] = test['year'].astype("object")

<font color="#8fce00"><h4><u><b>Note:</b></u></h4></font> 
Converting the <font color="#da70d6"><b>date</b></font> feature into proper datetime format.

In [ ]:
train['date'] = pd.to_datetime(arg=train['date'])
test['date'] = pd.to_datetime(arg=test['date'])

In [ ]:
columns_categorical = []
for col in train.columns:
  if train[col].dtype=="object":
    columns_categorical.append(col)
print("No. of categorical features:",len(columns_categorical))    

No. of categorical features: 25


In [ ]:
# sanity:-
print("train shape:",train.shape)
print("test shape:",test.shape)

train shape: (1708337, 44)
test shape: (401589, 44)


# <font color="#1b5776">3. <u>Data Preprocessing</u></font>

## <font color="#1b5776">3.1. <u>Preprocessing the Categorical Features</u></font>

<font color="#8fce00"><h4><u><b>Note:</b></u></h4></font> 
We have over 17 million data points and 44 features till now, out of which 25 are categorical. Some categories even have over 1000 different values. So, OHE will lead to a huge explosion of memory and hence, not a good idea.<br>
Instead, let's custom encode these features in such a way that that the category which leads to highest revenue generation will be encoded with the highest value.<br>
Eg: <font color="#da70d6"><b>channelGrouping</b></font> has 8 different categories in total. We've seen in EDA that maximum revenue is generated via <font color="#da70d6"><b>Organic Search</b></font>. It pretty much makes sense to encode it with 7 and so on with other categories.<br>
Note that this technique will make our features ordinal.<br>
Now, what advantage are we getting by doing this, instead of simple and library based Label Encoding or Ordinal Encoding (because they also perform same kind of operation)?<br>
It is because: Label/Ordinal Encoding will assign integer values randomly and without any special meaning. It's definitely better to perform custom encoding the way we decided instead (Atleast the encoding will have some meaning for being ordinal, one way or the other).

In [ ]:
def create_custom_encoding_dict(df):
  """
  This function is like fit method.
  It takes a dataframe and returns a dict of its features custom encoded in an order of decreading revenue generated via each value of each feature.
  """
  parent_CustomEncodingDict = {} # will contain 1 dictionary for each categorical feature in the dataframe df.

  for col in df.columns:
    child_CustomEncodingDict = {} # will contain custom encoded category values for feature --> col.
    if df[col].dtype == "object": # checking if the feature is categorical.
      # arranging each category value in a descending order of revenue generated.
      temp = df.groupby(by=col)["totals_transactionRevenue"].sum().sort_values(ascending=False)

      # eg: len(temp) --> is the no. of unique categories of a feature.
      # Hence, if no. of unique categories=8, then we want encoding as 7,6,5,4,3,2,1,0 (and not as 8,7,6,5,4,3,2,1).
      # That's why we'll take len(temp) - 1.
      encode = len(temp) - 1
      for cat in temp.index:
        child_CustomEncodingDict[cat] = encode
        encode = encode - 1
      parent_CustomEncodingDict[col] = child_CustomEncodingDict
  return parent_CustomEncodingDict

# now we call create_custom_encoding_dict function on train dataframe.
# NOTE: this is almost like calling "fit" method on train dataframe for encoding categorical features.
CUSTOM_ENCODING_DICT = create_custom_encoding_dict(train)

In [ ]:
# https://docs.python.org/2/library/collections.html#:~:text=Ordered%20dictionaries%20are%20just%20like%20regular%20dictionaries%20but%20they%20remember%20the%20order%20that%20items%20were%20inserted.%20When%20iterating%20over%20an%20ordered%20dictionary%2C%20the%20items%20are%20returned%20in%20the%20order%20their%20keys%20were%20first%20added.

# arranging every child dictionary in descending order of their encoding values (just for convenience of sanity viewing):-
for d in CUSTOM_ENCODING_DICT.values():
  d = OrderedDict(sorted(d.items(), key=lambda x:x[1], reverse=True))
print(CUSTOM_ENCODING_DICT)

{'channelGrouping': {'Organic Search': 7, 'Social': 6, 'Direct': 5, 'Referral': 4, 'Paid Search': 3, 'Affiliates': 2, 'Display': 1, '(Other)': 0}, 'device_browser': {'Chrome': 21, 'Safari': 20, 'Internet Explorer': 19, 'YaBrowser': 18, 'Amazon Silk': 17, 'Android Browser': 16, 'Android Webview': 15, 'BlackBerry': 14, 'Coc Coc': 13, 'Edge': 12, 'Firefox': 11, 'MRCHROME': 10, 'UC Browser': 9, 'Maxthon': 8, 'Mozilla Compatible Agent': 7, 'Opera': 6, 'Opera Mini': 5, 'Playstation Vita Browser': 4, 'Puffin': 3, 'Safari (in-app)': 2, 'Samsung Internet': 1, 'ADM': 0}, 'device_operatingSystem': {'Macintosh': 13, 'Windows': 12, 'Chrome OS': 11, 'iOS': 10, 'Linux': 9, 'Android': 8, 'Xbox': 7, 'Windows Phone': 6, 'Tizen': 5, 'Samsung': 4, 'Playstation Vita': 3, 'OTHERS': 2, 'OS/2': 1, 'BlackBerry': 0}, 'device_isMobile': {'No': 1, 'Yes': 0}, 'device_deviceCategory': {'desktop': 2, 'mobile': 1, 'tablet': 0}, 'geoNetwork_continent': {'Americas': 5, 'Asia': 4, 'Oceania': 3, 'OTHERS': 2, 'Europe': 1,

In [ ]:
# saving this dict to re-use again while creating production pipeline
f_path = "/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/CUSTOM_ENCODING_DICT.pkl"
with open(f_path,"wb") as f:
  pickle.dump(CUSTOM_ENCODING_DICT,f)
f.close()  

In [ ]:
def perform_custom_encoding(x,feature_specfic_dict):
  """This is like transform method. It will take a value -->x and encode it using the feature-specific dictionary passed here."""
  if x in feature_specfic_dict.keys():
    return feature_specfic_dict[x]
  else:
    return 0 # this 0 is for those feature values which wasn't seen during the fit method in train dataset

for feature in tqdm(CUSTOM_ENCODING_DICT.keys()):
  train[feature] = train[feature].apply(perform_custom_encoding, args=(CUSTOM_ENCODING_DICT[feature],))
  test[feature] = test[feature].apply(perform_custom_encoding, args=(CUSTOM_ENCODING_DICT[feature],))    

100%|██████████| 25/25 [00:38<00:00,  1.53s/it]


In [ ]:
train.head(2)

,fullVisitorId,channelGrouping,date,visitNumber,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_networkDomain,totals_hits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_transactions,totals_transactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_referralPath,trafficSource_adContent,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.adNetworkType,value,year,month,dayOfMonth,dayOfWeek,weekOfYear,dayOfYear,quarter,isYearEnd,is_weekend,visit_hr,visit_min,visit_sec,time_of_day,is_office_hours
0,3162355547410993243,7,2017-10-16,1,11,12,1,2,1,16,124,94,26,119,1349,1,1.0,1.0,0.0,0.0,0.0,10,50,4,15,242,14,0.0,2,3,2,10,16,0,42,289,4,0,1,0,0,50,1,1
1,8934116514970143966,4,2017-10-16,6,21,11,1,2,5,18,130,93,25,81,1349,2,2.0,2.0,28.0,0.0,0.0,10,49,2,57,167,14,0.0,2,5,2,10,16,0,42,289,4,0,1,17,51,47,3,0


In [ ]:
test.head(2)

,fullVisitorId,channelGrouping,date,visitNumber,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_networkDomain,totals_hits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_transactions,totals_transactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_referralPath,trafficSource_adContent,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.adNetworkType,value,year,month,dayOfMonth,dayOfWeek,weekOfYear,dayOfYear,quarter,isYearEnd,is_weekend,visit_hr,visit_min,visit_sec,time_of_day,is_office_hours
0,7460955084541987166,7,2018-05-11,2,21,8,0,1,4,3,109,65,26,119,1349,4,3.0,1,973.0,0.0,0.0,10,50,4,57,242,14,0.0,2,2,0,5,11,4,19,131,2,1,1,4,29,1,1,1
1,460252456180441002,5,2018-05-11,166,21,13,1,2,5,18,130,93,25,114,1349,4,3.0,1,49.0,0.0,0.0,10,51,5,57,242,14,0.0,2,5,0,5,11,4,19,131,2,1,1,18,48,3,2,1


## <font color="#1b5776">3.2. <u>Leveraging the Time Based Angle for next level of Preprocessing</u></font>

<font color="#8fce00"><h4><u><b>What we have:-</b></u></h4></font>
* We are provided with a <font color="#b00be5">test dataset (test_v2.csv)</font> which contains data from <font color="#b00be5">1st May,2018 to 15th Oct,2018 (spanning 168 days in total)</font> and the <font color="#b00be5">prediction dataset (submission_v2.csv)</font> containing visits from <font color="#b00be5">1st Dec,2018 to 31st Jan,2019 (spanning 62 days in total)</font> after a <font color="#b00be5">gap period of 46 days (from 16th Oct,2018 to 30th Nov,2018)</font> for which we need to predict the revenues.
* Basically the submission dataset has some visitors who also visited the Gstore site in the test dataset’s period of 168 days earlier.

<font color="#8fce00"><h4><u><b>How can we leverage such kind of time-frame based available data:-</b></u></h4></font>
* I’ll split the train and test data’s timelines into consecutive time-periods, keeping in mind that my model needs to predict the revenue for the future time-period.
* The <font color="#b00be5">train</font> data is from <font color="#b00be5">1st Aug, 2016 to 30th April, 2018</font>. And the <font color="#b00be5">test</font> data is from <font color="#b00be5">1st May, 2018 to 15th Oct, 2018(168 days)</font>. There is a <font color="#b00be5">gap period of 46 days</font> from <font color="#b00be5">16th Oct, 2018 to 30th Nov, 2018</font> after which starts the time period of our <font color="#b00be5">submission_v2.csv (1st Dec, 2018 to 31st Jan,2019)</font> for which we need to predict the revenues generated by each visitor.
* <font color="#b00be5">Our task here</font> → is to <b>predict the revenues generated by the visitors in submission_v2.csv</b> using the <font color="#b00be5">visitor data in test_v2.csv as test set and train_v2.csv as train set</font>. Thus, we are going to <b>predict the future using historical data</b>.
* I’ll replicate this same thing for creating the train dataset as well. I’ll <font color="#b00be5">create a 168 day time-frame and collect all data-points falling in this time frame</font>. Consecutively, I’ll <font color="#b00be5">keep a gap of 46 days in between and find all data-points falling in the 62 day time-frame after this 46 day gap</font>. I’ll repeat this until I create as many such possible time-frames as possible (basically there will be 4 such frames - each having 3 sub-frames - 168 + 46 + 62).
* Inspiration: [Winner of Google Analytics Customer Revenue Prediction](https://www.kaggle.com/c/ga-customer-revenue-prediction/discussion/82614)

<font color="#8fce00"><h4><u><b>Why should we do this:-</b></u></h4></font>
* Basically we have maximum visitors who don’t end up buying from the GStore, even after visiting it again and there are many visitors who don’t come back again as well.
* And hence, eliminating this “<b>HURDLE</b>” is important, the “<b>HURDLE</b>” being detecting whether the customer will at-least come back or not. And “<b>if the customer comes back</b>”, then we’ll predict the revenue that this customer may generate in this 62 day period. This way, <b>we are helping the model to understand the problem at hand in parts, so that it doesn’t “over learn” and predict for customers who won’t come back only!</b>
* There will be <b>2 target variables now: one for Classification model and another for Regression model</b>.
* <b>Target variable for the Classification model will be a simple indicator variable: 1 or 0</b> → wherein 1 means that the visitor returned in the 62 day period after the 46 day gap period after visiting the Gstore in the 168 day period just before this 46 day gap and 0 will simply mean the reverse (i.e. the visitor didn’t return and hence leading to no revenue).
* <b>Target variable for the Regression model will be the log of the (sum of revenues + 1) generated by each customer in that 62 day period, for the customers in the 168 day period</b>. 

### <font color="#8fce00"><u><b>Analysing train,test & submit timeframes:-</b></u></font>

In [ ]:
# sanity of time-frames available:-
train_start = train["date"].min()
train_end = train['date'].max()
print("In train:",train_start,"to",train_end)

test_start = test["date"].min()
test_end = test['date'].max()
print("In test:",test_start,"to",test_end)

submit_start = datetime.datetime(2018,12,1)
submit_end = datetime.datetime(2019,1,31)
print("In submit:",submit_start,"to",submit_end)

In train: 2016-08-01 00:00:00 to 2018-04-30 00:00:00
In test: 2018-05-01 00:00:00 to 2018-10-15 00:00:00
In submit: 2018-12-01 00:00:00 to 2019-01-31 00:00:00


In [ ]:
# +1 to include 1 removed day (the test_start day) (since we want the total no. of days that the test data spans, including its start and its end day)
test_span = (test_end - test_start).days + 1

# +1 to include 1 removed day (the submit_start day) (since we want the total no. of days that the submit data spans, including its start and its end day)
submit_span = (submit_end - submit_start).days + 1

# -1 to remove 1 included day (the test_start day) (since we want the number of days "BETWEEN" test_end & submit_start)
gap = (submit_start - test_end).days - 1

print("test set spans",test_span,"days")
print("submit set spans",submit_span,"days")
print("Gap period b/w test end and submit start is",gap,"days")

test set spans 168 days
submit set spans 62 days
Gap period b/w test end and submit start is 46 days


<font color="#8fce00"><h4><u><b>Note:</b></u></h4></font>  We have non-overlapping time-spans in our train,test as well as submit.
We are required to predict for the future time period of submit data --> 1st Dec,2018 to 31st Jan,2019.

### <font color="#8fce00"><u><b>Get dates for different timeframes:-</b></u></font>

In [ ]:
def get_splits(date):
  """Takes a date and returns 4 dates spanning across some days in between each other"""
  pre_gap_start = date

  # -1 to remove 1 included day (the pre_gap_start) (since we want 168 days INCLUDING the pre_gap_start and pre_gap_end)
  pre_gap_end = pre_gap_start + datetime.timedelta(test_span-1)

  # +1 to ensure there are exactly 46 days between pre_gap_end date and post_gap_start date
  post_gap_start = pre_gap_end + datetime.timedelta(gap+1)

  # -1 to remove 1 included day (the post_gap_start) (since we want 62 days INCLUDING the post_gap_start and post_gap_end)
  post_gap_end = post_gap_start + datetime.timedelta(submit_span-1)

  return pre_gap_start,pre_gap_end,post_gap_start,post_gap_end

split_1_pre_gap_start,split_1_pre_gap_end,split_1_post_gap_start,split_1_post_gap_end = get_splits(train_start)
print("split_1_pre_gap_start=",split_1_pre_gap_start)
print("split_1_pre_gap_end=",split_1_pre_gap_end)
print("split_1_post_gap_start=",split_1_post_gap_start)
print("split_1_post_gap_end=",split_1_post_gap_end)
print()

# adding +1 to the previous pre-gap-period's end date to ensure obtaining non-overlapping time periods b/w one pre-gap-period and another
split_2_pre_gap_start,split_2_pre_gap_end,split_2_post_gap_start,split_2_post_gap_end = get_splits(split_1_pre_gap_end+datetime.timedelta(1))
print("split_2_pre_gap_start=",split_2_pre_gap_start)
print("split_2_pre_gap_end=",split_2_pre_gap_end)
print("split_2_post_gap_start=",split_2_post_gap_start)
print("split_2_post_gap_end=",split_2_post_gap_end)
print()

# adding +1 to the previous pre-gap-period's end date to ensure obtaining non-overlapping time periods b/w one pre-gap-period and another
split_3_pre_gap_start,split_3_pre_gap_end,split_3_post_gap_start,split_3_post_gap_end = get_splits(split_2_pre_gap_end+datetime.timedelta(1))
print("split_3_pre_gap_start=",split_3_pre_gap_start)
print("split_3_pre_gap_end=",split_3_pre_gap_end)
print("split_3_post_gap_start=",split_3_post_gap_start)
print("split_3_post_gap_end=",split_3_post_gap_end)
print()

# adding +1 to the previous pre-gap-period's end date to ensure obtaining non-overlapping time periods b/w one pre-gap-period and another
split_4_pre_gap_start,split_4_pre_gap_end,split_4_post_gap_start,split_4_post_gap_end = get_splits(split_3_pre_gap_end+datetime.timedelta(1))
print("split_4_pre_gap_start=",split_4_pre_gap_start)
print("split_4_pre_gap_end=",split_4_pre_gap_end)
print("split_4_post_gap_start=",split_4_post_gap_start)
print("split_4_post_gap_end=",split_4_post_gap_end)

split_1_pre_gap_start= 2016-08-01 00:00:00
split_1_pre_gap_end= 2017-01-15 00:00:00
split_1_post_gap_start= 2017-03-03 00:00:00
split_1_post_gap_end= 2017-05-03 00:00:00

split_2_pre_gap_start= 2017-01-16 00:00:00
split_2_pre_gap_end= 2017-07-02 00:00:00
split_2_post_gap_start= 2017-08-18 00:00:00
split_2_post_gap_end= 2017-10-18 00:00:00

split_3_pre_gap_start= 2017-07-03 00:00:00
split_3_pre_gap_end= 2017-12-17 00:00:00
split_3_post_gap_start= 2018-02-02 00:00:00
split_3_post_gap_end= 2018-04-04 00:00:00

split_4_pre_gap_start= 2017-12-18 00:00:00
split_4_pre_gap_end= 2018-06-03 00:00:00
split_4_post_gap_start= 2018-07-20 00:00:00
split_4_post_gap_end= 2018-09-19 00:00:00


In [ ]:
train_plus_test = train.append(test) # to get insights on the customers who fall in split_4
print(train_plus_test.shape)
print(train.shape[0]+test.shape[0]) # sanity

(2109926, 44)
2109926


In [ ]:
# https://stackoverflow.com/questions/36921951/truth-value-of-a-series-is-ambiguous-use-a-empty-a-bool-a-item-a-any-o
# https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values

In [ ]:
def get_dates(pre_gap_start_date, pre_gap_end_date, post_gap_start_date, post_gap_end_date, data=train_plus_test):
  """
  This function filters and returns series of dates which fall b/w start_date & end_date (both inclusive).
  """
  # pre_gap_period --> timeframe before the 46-day gap.
  pre_gap_period = data['date'][ (data['date']>=pre_gap_start_date) & (data['date']<=pre_gap_end_date) ]

  # post_gap_period --> timeframe after the 46-day gap.
  post_gap_period = data['date'][ (data['date']>=post_gap_start_date) & (data['date']<=post_gap_end_date) ]
  
  return pre_gap_period,post_gap_period

In [ ]:
# timeframe-1
period_1_pre_gap,period_1_post_gap = get_dates(split_1_pre_gap_start,split_1_pre_gap_end,
                                                split_1_post_gap_start,split_1_post_gap_end)
# timeframe-2
period_2_pre_gap,period_2_post_gap = get_dates(split_2_pre_gap_start,split_2_pre_gap_end,
                                               split_2_post_gap_start,split_2_post_gap_end)
# timeframe-3
period_3_pre_gap,period_3_post_gap = get_dates(split_3_pre_gap_start,split_3_pre_gap_end,
                                               split_3_post_gap_start,split_3_post_gap_end)
# timeframe-4
period_4_pre_gap,period_4_post_gap = get_dates(split_4_pre_gap_start,split_4_pre_gap_end,
                                               split_4_post_gap_start,split_4_post_gap_end)

In [ ]:
# sanity
print(period_1_pre_gap.min(),",",period_1_pre_gap.max())
print(period_1_post_gap.min(),",",period_1_post_gap.max())
print()
print(period_2_pre_gap.min(),",",period_2_pre_gap.max())
print(period_2_post_gap.min(),",",period_2_post_gap.max())
print()
print(period_3_pre_gap.min(),",",period_3_pre_gap.max())
print(period_3_post_gap.min(),",",period_3_post_gap.max())
print()
print(period_4_pre_gap.min(),",",period_4_pre_gap.max())
print(period_4_post_gap.min(),",",period_4_post_gap.max())

2016-08-01 00:00:00 , 2017-01-15 00:00:00
2017-03-03 00:00:00 , 2017-05-03 00:00:00

2017-01-16 00:00:00 , 2017-07-02 00:00:00
2017-08-18 00:00:00 , 2017-10-18 00:00:00

2017-07-03 00:00:00 , 2017-12-17 00:00:00
2018-02-02 00:00:00 , 2018-04-04 00:00:00

2017-12-18 00:00:00 , 2018-06-03 00:00:00
2018-07-20 00:00:00 , 2018-09-19 00:00:00


### <font color="#8fce00"><u><b>Split main dataframes into different timeframes:-</b></u></font>

In [ ]:
def getSplits_ReturnedAndRevenueDetails(pre_gap_period,post_gap_period,df=train_plus_test):
  """Takes time periods, filters df using the timeframes, aggregates data and returns 2 dataframes."""

  # part of dataframe falling in pre-gap-period.
  pre_gap_df = df[ df['date'].isin(pre_gap_period) ]

  # part of dataframe falling in post-gap-period.
  post_gap_df = df[ df['date'].isin(post_gap_period) ]

  # set of customers who visited in the 168-day period and returned in the 62-day period after the 46-day gap.
  returned_post_gap = set(pre_gap_df["fullVisitorId"]).intersection(set(post_gap_df["fullVisitorId"]))

  # set of customers who visited in the 168-day period, but didn't return after the 46-day gap.
  not_returned_post_gap = pd.Series( list(set(pre_gap_df["fullVisitorId"]) - returned_post_gap) )
  
  # https://numpy.org/doc/stable/reference/generated/numpy.where.html
  # AIM: to create a new feature --> returnedPostGap --> of 1s and 0s.
  # 1 --> visitor from pre-gap period returned in post-gap period; 0 --> otherwise.

  # 1. create an indicator variable --> here, a pandas series of 1s and 0s.
  detect_return = pd.Series(np.where(pre_gap_df["fullVisitorId"].isin(returned_post_gap),1,0))

  # 2. create a dataframe of visitor ids and an indicator variable --> detect_return (having 1/0).
  # NOTE: This is a visit/session-level dataframe.
  detect_return_df = pd.DataFrame(data={"fullVisitorId":pre_gap_df["fullVisitorId"].values,"returnedPostGap":detect_return.values})

  # 3. group data by visitor.
  # eg: detect_return_df has 1 for all visits of a visitor who returned and 0 for all visits of a visitor who didn't return.
  # There's no point in keeping separate rows in which all rows will ultimately indicate the same thing.
  # Let's just group the data here in terms of visitors.
  # Hence, let's just take a simple mean (although, we can perform other operations like mode also).
  # eg: If a visitor has visited 5 times in pre-gap period and also returned in the post-gap period, then we'll have five 1s for this visitor.
  # We'll just take simple mean: (1+1+1+1+1)/5 which will simply be = 1.
  detect_return_df_visitorLevel = detect_return_df.groupby(by="fullVisitorId")["returnedPostGap"].mean().reset_index()
  
  # Find sum of all revenues for customers in post-gap period --> visitor-level data.
  # NOTE: This dataframe contains visitors who didn't visit before the 46-day gap as well.
  postGapCustomersRevenueSum = post_gap_df.groupby(by="fullVisitorId")["totals_transactionRevenue"].sum().reset_index()

  # Take log of (summed revenue + 1) --> adding 1 to handle zero revenues
  postGapCustomersRevenueSum["totals_transactionRevenue"] = postGapCustomersRevenueSum["totals_transactionRevenue"].apply(np.log1p)

  # Filtering those customers who also visited before the 46-day gap.
  # NOTE: This is the set of customers who matter the most!
  returnedCustomersRevenueSum = postGapCustomersRevenueSum[ postGapCustomersRevenueSum['fullVisitorId'].isin(returned_post_gap) ]

  # Setting revenue for customers who didn't return --> to zero.
  notReturnedCustomersRevenueSum = pd.DataFrame(data={"fullVisitorId":not_returned_post_gap.values,"totals_transactionRevenue":0})

  # concatenating the revenue info for all customers in pre-gap period.
  revenueDetailsAll_df = pd.concat([returnedCustomersRevenueSum,notReturnedCustomersRevenueSum],axis=0,ignore_index=True)
  # renaming the column for convenience.
  revenueDetailsAll_df = revenueDetailsAll_df.rename(columns={"totals_transactionRevenue":"totals_transactionRevenue_LogOfSum"})
  
  # Merged dataframe containing info on return as well as revenue generated.
  # NOTE: this is visitor-evel dataframe.
  totalRevenue_and_detectReturn_df = pd.merge(left=detect_return_df_visitorLevel,right=revenueDetailsAll_df,how="left",on="fullVisitorId")
  
  # returning the pre-gap dataframe to create aggregations on other features as well (SINCE CONVERTING INTO VISITOR-LEVEL IS OUR ULTIMATE GOAL).
  # SINCE, till now we've converted 2 columns to visitor-level: revenue and returnedPostGap.
  pre_gap_df = pre_gap_df.drop(columns=['totals_transactionRevenue'],axis=1).reset_index(drop=True)
  return pre_gap_df,totalRevenue_and_detectReturn_df    # pre_gap_df --> session-level df, totalRevenue_and_detectReturn_df --> visitor-level df

In [ ]:
# obtaning 4 splits and corresponding dataframes:-
split_1_pre_gap_df,split_1_totalRevenue_and_detectReturn_df = getSplits_ReturnedAndRevenueDetails(period_1_pre_gap,period_1_post_gap)
split_2_pre_gap_df,split_2_totalRevenue_and_detectReturn_df = getSplits_ReturnedAndRevenueDetails(period_2_pre_gap,period_2_post_gap)
split_3_pre_gap_df,split_3_totalRevenue_and_detectReturn_df = getSplits_ReturnedAndRevenueDetails(period_3_pre_gap,period_3_post_gap)
split_4_pre_gap_df,split_4_totalRevenue_and_detectReturn_df = getSplits_ReturnedAndRevenueDetails(period_4_pre_gap,period_4_post_gap)

# NOTE: Here, first dataframes are: visit/session-level; second dataframes are: visitor-level.
print(split_1_pre_gap_df.shape,",",split_1_totalRevenue_and_detectReturn_df.shape)
print(split_2_pre_gap_df.shape,",",split_2_totalRevenue_and_detectReturn_df.shape)
print(split_3_pre_gap_df.shape,",",split_3_totalRevenue_and_detectReturn_df.shape)
print(split_4_pre_gap_df.shape,",",split_4_totalRevenue_and_detectReturn_df.shape)

(465858, 43) , (377186, 3)
(367370, 43) , (288869, 3)
(497920, 43) , (385318, 3)
(471114, 43) , (366202, 3)


In [ ]:
split_1_pre_gap_df.head(2)

,fullVisitorId,channelGrouping,date,visitNumber,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_networkDomain,totals_hits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_transactions,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_referralPath,trafficSource_adContent,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.adNetworkType,value,year,month,dayOfMonth,dayOfWeek,weekOfYear,dayOfYear,quarter,isYearEnd,is_weekend,visit_hr,visit_min,visit_sec,time_of_day,is_office_hours
0,9499785259412240342,7,2016-09-02,1,11,12,1,2,1,16,29,94,26,119,670,1,1.0,0.0,0.0,0.0,10,50,4,57,242,14,0.0,2,3,1,9,2,4,35,246,3,1,1,10,31,12,0,0
1,3696906537737368442,7,2016-09-02,1,21,12,1,2,5,5,91,54,26,62,62,1,1.0,0.0,0.0,0.0,10,50,4,57,242,14,0.0,2,1,1,9,2,4,35,246,3,1,1,22,54,34,2,1


In [ ]:
split_1_totalRevenue_and_detectReturn_df.head(2)

,fullVisitorId,returnedPostGap,totals_transactionRevenue_LogOfSum
0,0000010278554503158,0,0.0
1,0000020424342248747,0,0.0


### <font color="#8fce00"><u><b>Aggregating features on Visitor Level by performing some suitable functions:-</b></u></font>

#### <font color="#b00be5"><b>Defining all necessary functions for Visitor-Level Feature Engineering</b></font>

In [ ]:
# https://stackoverflow.com/questions/44635626/rename-result-columns-from-pandas-aggregation-futurewarning-using-a-dict-with#:~:text=For%20each%20column,function%20to%20apply%27)%5D%7D)
# https://pbpython.com/groupby-agg.html
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html

In [ ]:
def aggApply_ModeMax(x):
  """
  eg-1: For browser feature:if customer visited 3 times via Chrome and never via any other browser, then return 3.
  eg-2: For weekend feature: if customer visited 2 times on weekdays and 2 times on weekends, i.e. we'll have two 1s and two 0s --> then return 1.
        Max is taken for this kind of case just to select 1 value. Note: Taking max/min won't affect the actual mode obtained.
  """
  return x.mode().max()

def agg_apply_Median(x):
  """return median of all values"""
  return x.median()

def agg_apply_Mean(x):
  """return mean of all values"""
  return x.mean()

def agg_apply_Sum(x):
  """return sum of all values"""
  return x.mean()

def agg_apply_Min(x):
  """return minimum of all values"""
  return x.min()

def agg_apply_Max(x):  
  """return max of all values"""
  return x.max()

def agg_apply_LogOfSum(x):
  """return log of (sum of all values + 1). NOTE: adding 1 just to handle 0 sum."""
  return np.log1p(np.sum(x))

def agg_apply_SpanVisits(x):
  """returns no. of days in between 1st visit & last visit in this particular aggregated time-frame"""
  return (x.max() - x.min()).days

In [ ]:
# all categorical features:-
set1 = columns_categorical

# all int/float features on which "aggApply_ModeMax" function to be applied:-
set2 = ["dayOfMonth","dayOfWeek","dayOfYear","month","quarter","visitNumber","visit_hr","weekOfYear"]

# all other features in each timeframe-based dataframe:-
set3 = list ((set(split_1_pre_gap_df.columns) - set(columns_categorical)) - set(["fullVisitorId","date"]) )

# date column:-
set4 = ["date"]

# revenue column (will be present in test dataframe only; since we've already preprocessed revenue columns of the 4 splits):-
set5 = ["totals_transactionRevenue"]

In [ ]:
# saving this list to re-use again while creating production pipeline
f_path = "/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/set1.pkl"
with open(f_path,"wb") as f:
  pickle.dump(set1,f)
f.close()

# saving this list to re-use again while creating production pipeline
f_path = "/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/set2.pkl"
with open(f_path,"wb") as f:
  pickle.dump(set2,f)
f.close()

# saving this list to re-use again while creating production pipeline
f_path = "/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/set3.pkl"
with open(f_path,"wb") as f:
  pickle.dump(set3,f)
f.close()

# saving this list to re-use again while creating production pipeline
f_path = "/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/set4.pkl"
with open(f_path,"wb") as f:
  pickle.dump(set4,f)
f.close()

# saving this list to re-use again while creating production pipeline
f_path = "/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/set5.pkl"
with open(f_path,"wb") as f:
  pickle.dump(set5,f)
f.close()

In [ ]:
def get_apply_dict(df):
  """Returns a dictionary with feature name and list of tuples of new aggregated feature name & aggregation functions to apply."""
  apply_dict = {}

  for col in df.columns:
    if (col in set1) or (col in set2):
      new_feature_name = col+"_ModeMax"
      apply_dict[col] = [(new_feature_name, aggApply_ModeMax)]

    if col in set3:
      new_feature_name1 = col+"_Median"
      new_feature_name2 = col+"_Mean"
      new_feature_name3 = col+"_Sum"
      new_feature_name4 = col+"_Min"
      new_feature_name5 = col+"_Max"
      apply_list = [(new_feature_name1, agg_apply_Median), (new_feature_name2, agg_apply_Mean),
                    (new_feature_name3, agg_apply_Sum), (new_feature_name4, agg_apply_Min),
                    (new_feature_name5, agg_apply_Max)]
      if col in apply_dict.keys(): # checking if this col is already present in the dict
        apply_dict[col].extend(apply_list)
      else:
        apply_dict[col] = apply_list

    if col in set4:
      new_feature_name1 = col+"_Span"
      new_feature_name2 = col+"_FirstVisit"
      new_feature_name3 = col+"_LastVisit"
      apply_dict[col] = [(new_feature_name1, agg_apply_SpanVisits),(new_feature_name2, "min"),
                         (new_feature_name3, "max")]

    if col in set5:
      new_feature_name = col+"_LogOfSum"
      apply_dict[col] = [(new_feature_name, agg_apply_LogOfSum)]

  return apply_dict

def vistor_level_data(data):
  """Main function to get visitor-level data for the fed dataframe"""
  data = data.groupby(by="fullVisitorId").agg(get_apply_dict(data))
  return data

In [ ]:
def firstVisitAfterStart(x,start):
  """No. of days after current period's start date, the First visit occurred"""
  return (x - start).days

def lastVisitBeforeEnd(x,end):
  """No. of days before current period's end date, the Last visit occurred"""
  return (end - x).days

In [ ]:
def apply_to_splits(split_df,current_period_start,current_period_end):
  """Applies function to 2 date columns and returns the modified dataframe"""
  split_df["firstVisit_AfterStart"] = split_df["date_FirstVisit"].apply(firstVisitAfterStart, args=(current_period_start,))
  split_df["lastVisit_BeforeEnd"] = split_df["date_LastVisit"].apply(lastVisitBeforeEnd, args=(current_period_end,))
  split_df = split_df.drop(columns=["date_FirstVisit","date_LastVisit"], axis=1)
  return split_df

#### <font color="#b00be5"><b>Split - 1</b></font>

In [ ]:
%%time
split_1_visitorLevel = vistor_level_data(split_1_pre_gap_df)

CPU times: user 1h 3min 7s, sys: 2min 45s, total: 1h 5min 52s
Wall time: 1h 2min 35s


In [ ]:
split_1_visitorLevel.shape

(377186, 116)

In [ ]:
# dropping the multi-index level, resetting index and set fullVisitorId as the 1st column.
split_1_visitorLevel.columns = split_1_visitorLevel.columns.droplevel()
split_1_visitorLevel = split_1_visitorLevel.reset_index()

In [ ]:
split_1_visitorLevel.shape

(377186, 117)

In [ ]:
# merge revenue and returned data with other features.
split_1 = pd.merge(left=split_1_visitorLevel,right=split_1_totalRevenue_and_detectReturn_df,how="left",on="fullVisitorId")
split_1.shape

(377186, 119)

In [ ]:
split_1.head(2)

,fullVisitorId,channelGrouping_ModeMax,date_Span,date_FirstVisit,date_LastVisit,visitNumber_ModeMax,visitNumber_Median,visitNumber_Mean,visitNumber_Sum,visitNumber_Min,visitNumber_Max,device_browser_ModeMax,device_operatingSystem_ModeMax,device_isMobile_ModeMax,device_deviceCategory_ModeMax,geoNetwork_continent_ModeMax,geoNetwork_subContinent_ModeMax,geoNetwork_country_ModeMax,geoNetwork_region_ModeMax,geoNetwork_metro_ModeMax,geoNetwork_city_ModeMax,geoNetwork_networkDomain_ModeMax,totals_hits_Median,totals_hits_Mean,totals_hits_Sum,totals_hits_Min,totals_hits_Max,totals_pageviews_Median,totals_pageviews_Mean,totals_pageviews_Sum,totals_pageviews_Min,totals_pageviews_Max,totals_sessionQualityDim_Median,totals_sessionQualityDim_Mean,totals_sessionQualityDim_Sum,totals_sessionQualityDim_Min,totals_sessionQualityDim_Max,totals_timeOnSite_Median,totals_timeOnSite_Mean,totals_timeOnSite_Sum,...,weekOfYear_ModeMax,weekOfYear_Median,weekOfYear_Mean,weekOfYear_Sum,weekOfYear_Min,weekOfYear_Max,dayOfYear_ModeMax,dayOfYear_Median,dayOfYear_Mean,dayOfYear_Sum,dayOfYear_Min,dayOfYear_Max,quarter_ModeMax,quarter_Median,quarter_Mean,quarter_Sum,quarter_Min,quarter_Max,isYearEnd_ModeMax,is_weekend_ModeMax,visit_hr_ModeMax,visit_hr_Median,visit_hr_Mean,visit_hr_Sum,visit_hr_Min,visit_hr_Max,visit_min_Median,visit_min_Mean,visit_min_Sum,visit_min_Min,visit_min_Max,visit_sec_Median,visit_sec_Mean,visit_sec_Sum,visit_sec_Min,visit_sec_Max,time_of_day_ModeMax,is_office_hours_ModeMax,returnedPostGap,totals_transactionRevenue_LogOfSum
0,0000010278554503158,7,0,2016-10-20,2016-10-20,1,1.0,1.0,1.0,1,1,21,13,1,2,3,0,15,94,26,119,406,3.0,3.0,3.0,3,3,3.0,3.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,61.0,61.0,61.0,...,42,42.0,42.0,42.0,42,42,294,294.0,294.0,294.0,294,294,4,4.0,4.0,4.0,4,4,0,1,5,5.0,5.0,5.0,5,5,57.0,57.0,57.0,57,57,46.0,46.0,46.0,46,46,1,1,0,0.0
1,0000020424342248747,7,0,2016-11-30,2016-11-30,1,1.0,1.0,1.0,1,1,20,10,0,1,4,17,47,61,26,74,1349,4.0,4.0,4.0,4,4,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,51.0,51.0,51.0,...,48,48.0,48.0,48.0,48,48,335,335.0,335.0,335.0,335,335,4,4.0,4.0,4.0,4,4,0,1,7,7.0,7.0,7.0,7,7,55.0,55.0,55.0,55,55,1.0,1.0,1.0,1,1,0,1,0,0.0


In [ ]:
split_1 = apply_to_splits(split_1, split_1_pre_gap_start, split_1_pre_gap_end)

In [ ]:
split_1.head(2)

,fullVisitorId,channelGrouping_ModeMax,date_Span,visitNumber_ModeMax,visitNumber_Median,visitNumber_Mean,visitNumber_Sum,visitNumber_Min,visitNumber_Max,device_browser_ModeMax,device_operatingSystem_ModeMax,device_isMobile_ModeMax,device_deviceCategory_ModeMax,geoNetwork_continent_ModeMax,geoNetwork_subContinent_ModeMax,geoNetwork_country_ModeMax,geoNetwork_region_ModeMax,geoNetwork_metro_ModeMax,geoNetwork_city_ModeMax,geoNetwork_networkDomain_ModeMax,totals_hits_Median,totals_hits_Mean,totals_hits_Sum,totals_hits_Min,totals_hits_Max,totals_pageviews_Median,totals_pageviews_Mean,totals_pageviews_Sum,totals_pageviews_Min,totals_pageviews_Max,totals_sessionQualityDim_Median,totals_sessionQualityDim_Mean,totals_sessionQualityDim_Sum,totals_sessionQualityDim_Min,totals_sessionQualityDim_Max,totals_timeOnSite_Median,totals_timeOnSite_Mean,totals_timeOnSite_Sum,totals_timeOnSite_Min,totals_timeOnSite_Max,...,weekOfYear_Mean,weekOfYear_Sum,weekOfYear_Min,weekOfYear_Max,dayOfYear_ModeMax,dayOfYear_Median,dayOfYear_Mean,dayOfYear_Sum,dayOfYear_Min,dayOfYear_Max,quarter_ModeMax,quarter_Median,quarter_Mean,quarter_Sum,quarter_Min,quarter_Max,isYearEnd_ModeMax,is_weekend_ModeMax,visit_hr_ModeMax,visit_hr_Median,visit_hr_Mean,visit_hr_Sum,visit_hr_Min,visit_hr_Max,visit_min_Median,visit_min_Mean,visit_min_Sum,visit_min_Min,visit_min_Max,visit_sec_Median,visit_sec_Mean,visit_sec_Sum,visit_sec_Min,visit_sec_Max,time_of_day_ModeMax,is_office_hours_ModeMax,returnedPostGap,totals_transactionRevenue_LogOfSum,firstVisit_AfterStart,lastVisit_BeforeEnd
0,0000010278554503158,7,0,1,1.0,1.0,1.0,1,1,21,13,1,2,3,0,15,94,26,119,406,3.0,3.0,3.0,3,3,3.0,3.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,61.0,61.0,61.0,61.0,61.0,...,42.0,42.0,42,42,294,294.0,294.0,294.0,294,294,4,4.0,4.0,4.0,4,4,0,1,5,5.0,5.0,5.0,5,5,57.0,57.0,57.0,57,57,46.0,46.0,46.0,46,46,1,1,0,0.0,80,87
1,0000020424342248747,7,0,1,1.0,1.0,1.0,1,1,20,10,0,1,4,17,47,61,26,74,1349,4.0,4.0,4.0,4,4,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,51.0,51.0,51.0,51.0,51.0,...,48.0,48.0,48,48,335,335.0,335.0,335.0,335,335,4,4.0,4.0,4.0,4,4,0,1,7,7.0,7.0,7.0,7,7,55.0,55.0,55.0,55,55,1.0,1.0,1.0,1,1,0,1,0,0.0,121,46


In [ ]:
split_1.to_csv("/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/split_1_visitor_level.csv",index=False)

#### <font color="#b00be5"><b>Split - 2</b></font>

In [ ]:
%%time
split_2_visitorLevel = vistor_level_data(split_2_pre_gap_df)

CPU times: user 48min 3s, sys: 2min 2s, total: 50min 5s
Wall time: 47min 39s


In [ ]:
split_2_visitorLevel.shape

(288869, 116)

In [ ]:
# dropping the multi-index level, resetting index and set fullVisitorId as the 1st column.
split_2_visitorLevel.columns = split_2_visitorLevel.columns.droplevel()
split_2_visitorLevel = split_2_visitorLevel.reset_index()

In [ ]:
split_2_visitorLevel.shape

(288869, 117)

In [ ]:
# merge revenue and returned data with other features.
split_2 = pd.merge(left=split_2_visitorLevel,right=split_2_totalRevenue_and_detectReturn_df,how="left",on="fullVisitorId")
split_2.shape

(288869, 119)

In [ ]:
split_2.head(2)

,fullVisitorId,channelGrouping_ModeMax,date_Span,date_FirstVisit,date_LastVisit,visitNumber_ModeMax,visitNumber_Median,visitNumber_Mean,visitNumber_Sum,visitNumber_Min,visitNumber_Max,device_browser_ModeMax,device_operatingSystem_ModeMax,device_isMobile_ModeMax,device_deviceCategory_ModeMax,geoNetwork_continent_ModeMax,geoNetwork_subContinent_ModeMax,geoNetwork_country_ModeMax,geoNetwork_region_ModeMax,geoNetwork_metro_ModeMax,geoNetwork_city_ModeMax,geoNetwork_networkDomain_ModeMax,totals_hits_Median,totals_hits_Mean,totals_hits_Sum,totals_hits_Min,totals_hits_Max,totals_pageviews_Median,totals_pageviews_Mean,totals_pageviews_Sum,totals_pageviews_Min,totals_pageviews_Max,totals_sessionQualityDim_Median,totals_sessionQualityDim_Mean,totals_sessionQualityDim_Sum,totals_sessionQualityDim_Min,totals_sessionQualityDim_Max,totals_timeOnSite_Median,totals_timeOnSite_Mean,totals_timeOnSite_Sum,...,weekOfYear_ModeMax,weekOfYear_Median,weekOfYear_Mean,weekOfYear_Sum,weekOfYear_Min,weekOfYear_Max,dayOfYear_ModeMax,dayOfYear_Median,dayOfYear_Mean,dayOfYear_Sum,dayOfYear_Min,dayOfYear_Max,quarter_ModeMax,quarter_Median,quarter_Mean,quarter_Sum,quarter_Min,quarter_Max,isYearEnd_ModeMax,is_weekend_ModeMax,visit_hr_ModeMax,visit_hr_Median,visit_hr_Mean,visit_hr_Sum,visit_hr_Min,visit_hr_Max,visit_min_Median,visit_min_Mean,visit_min_Sum,visit_min_Min,visit_min_Max,visit_sec_Median,visit_sec_Mean,visit_sec_Sum,visit_sec_Min,visit_sec_Max,time_of_day_ModeMax,is_office_hours_ModeMax,returnedPostGap,totals_transactionRevenue_LogOfSum
0,0000027376579751715,7,0,2017-02-11,2017-02-11,1,1.0,1.0,1.0,1,1,21,12,1,2,5,18,130,94,26,119,96,1.0,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6,6.0,6.0,6.0,6,6,42,42.0,42.0,42.0,42,42,1,1.0,1.0,1.0,1,1,1,0,2,2.0,2.0,2.0,2,2,24.0,24.0,24.0,24,24,53.0,53.0,53.0,53,53,1,1,0,0.0
1,0000039460501403861,6,0,2017-03-27,2017-03-27,1,1.0,1.0,1.0,1,1,21,12,1,2,4,11,55,22,26,39,1349,1.0,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13,13.0,13.0,13.0,13,13,86,86.0,86.0,86.0,86,86,1,1.0,1.0,1.0,1,1,1,1,15,15.0,15.0,15.0,15,15,45.0,45.0,45.0,45,45,16.0,16.0,16.0,16,16,3,0,0,0.0


In [ ]:
split_2 = apply_to_splits(split_2, split_2_pre_gap_start, split_2_pre_gap_end)

In [ ]:
split_2.head(2)

,fullVisitorId,channelGrouping_ModeMax,date_Span,visitNumber_ModeMax,visitNumber_Median,visitNumber_Mean,visitNumber_Sum,visitNumber_Min,visitNumber_Max,device_browser_ModeMax,device_operatingSystem_ModeMax,device_isMobile_ModeMax,device_deviceCategory_ModeMax,geoNetwork_continent_ModeMax,geoNetwork_subContinent_ModeMax,geoNetwork_country_ModeMax,geoNetwork_region_ModeMax,geoNetwork_metro_ModeMax,geoNetwork_city_ModeMax,geoNetwork_networkDomain_ModeMax,totals_hits_Median,totals_hits_Mean,totals_hits_Sum,totals_hits_Min,totals_hits_Max,totals_pageviews_Median,totals_pageviews_Mean,totals_pageviews_Sum,totals_pageviews_Min,totals_pageviews_Max,totals_sessionQualityDim_Median,totals_sessionQualityDim_Mean,totals_sessionQualityDim_Sum,totals_sessionQualityDim_Min,totals_sessionQualityDim_Max,totals_timeOnSite_Median,totals_timeOnSite_Mean,totals_timeOnSite_Sum,totals_timeOnSite_Min,totals_timeOnSite_Max,...,weekOfYear_Mean,weekOfYear_Sum,weekOfYear_Min,weekOfYear_Max,dayOfYear_ModeMax,dayOfYear_Median,dayOfYear_Mean,dayOfYear_Sum,dayOfYear_Min,dayOfYear_Max,quarter_ModeMax,quarter_Median,quarter_Mean,quarter_Sum,quarter_Min,quarter_Max,isYearEnd_ModeMax,is_weekend_ModeMax,visit_hr_ModeMax,visit_hr_Median,visit_hr_Mean,visit_hr_Sum,visit_hr_Min,visit_hr_Max,visit_min_Median,visit_min_Mean,visit_min_Sum,visit_min_Min,visit_min_Max,visit_sec_Median,visit_sec_Mean,visit_sec_Sum,visit_sec_Min,visit_sec_Max,time_of_day_ModeMax,is_office_hours_ModeMax,returnedPostGap,totals_transactionRevenue_LogOfSum,firstVisit_AfterStart,lastVisit_BeforeEnd
0,0000027376579751715,7,0,1,1.0,1.0,1.0,1,1,21,12,1,2,5,18,130,94,26,119,96,1.0,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,6.0,6,6,42,42.0,42.0,42.0,42,42,1,1.0,1.0,1.0,1,1,1,0,2,2.0,2.0,2.0,2,2,24.0,24.0,24.0,24,24,53.0,53.0,53.0,53,53,1,1,0,0.0,26,141
1,0000039460501403861,6,0,1,1.0,1.0,1.0,1,1,21,12,1,2,4,11,55,22,26,39,1349,1.0,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13.0,13.0,13,13,86,86.0,86.0,86.0,86,86,1,1.0,1.0,1.0,1,1,1,1,15,15.0,15.0,15.0,15,15,45.0,45.0,45.0,45,45,16.0,16.0,16.0,16,16,3,0,0,0.0,70,97


In [ ]:
split_2.to_csv("/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/split_2_visitor_level.csv",index=False)

#### <font color="#b00be5"><b>Split - 3</b></font>

In [ ]:
%%time
split_3_visitorLevel = vistor_level_data(split_3_pre_gap_df)

CPU times: user 1h 4min 47s, sys: 2min 35s, total: 1h 7min 22s
Wall time: 1h 4min 12s


In [ ]:
split_3_visitorLevel.shape

(385318, 116)

In [ ]:
# dropping the multi-index level, resetting index and set fullVisitorId as the 1st column.
split_3_visitorLevel.columns = split_3_visitorLevel.columns.droplevel()
split_3_visitorLevel = split_3_visitorLevel.reset_index()

In [ ]:
split_3_visitorLevel.shape

(385318, 117)

In [ ]:
# merge revenue and returned data with other features.
split_3 = pd.merge(left=split_3_visitorLevel,right=split_3_totalRevenue_and_detectReturn_df,how="left",on="fullVisitorId")
split_3.shape

(385318, 119)

In [ ]:
split_3.head(2)

,fullVisitorId,channelGrouping_ModeMax,date_Span,date_FirstVisit,date_LastVisit,visitNumber_ModeMax,visitNumber_Median,visitNumber_Mean,visitNumber_Sum,visitNumber_Min,visitNumber_Max,device_browser_ModeMax,device_operatingSystem_ModeMax,device_isMobile_ModeMax,device_deviceCategory_ModeMax,geoNetwork_continent_ModeMax,geoNetwork_subContinent_ModeMax,geoNetwork_country_ModeMax,geoNetwork_region_ModeMax,geoNetwork_metro_ModeMax,geoNetwork_city_ModeMax,geoNetwork_networkDomain_ModeMax,totals_hits_Median,totals_hits_Mean,totals_hits_Sum,totals_hits_Min,totals_hits_Max,totals_pageviews_Median,totals_pageviews_Mean,totals_pageviews_Sum,totals_pageviews_Min,totals_pageviews_Max,totals_sessionQualityDim_Median,totals_sessionQualityDim_Mean,totals_sessionQualityDim_Sum,totals_sessionQualityDim_Min,totals_sessionQualityDim_Max,totals_timeOnSite_Median,totals_timeOnSite_Mean,totals_timeOnSite_Sum,...,weekOfYear_ModeMax,weekOfYear_Median,weekOfYear_Mean,weekOfYear_Sum,weekOfYear_Min,weekOfYear_Max,dayOfYear_ModeMax,dayOfYear_Median,dayOfYear_Mean,dayOfYear_Sum,dayOfYear_Min,dayOfYear_Max,quarter_ModeMax,quarter_Median,quarter_Mean,quarter_Sum,quarter_Min,quarter_Max,isYearEnd_ModeMax,is_weekend_ModeMax,visit_hr_ModeMax,visit_hr_Median,visit_hr_Mean,visit_hr_Sum,visit_hr_Min,visit_hr_Max,visit_min_Median,visit_min_Mean,visit_min_Sum,visit_min_Min,visit_min_Max,visit_sec_Median,visit_sec_Mean,visit_sec_Sum,visit_sec_Min,visit_sec_Max,time_of_day_ModeMax,is_office_hours_ModeMax,returnedPostGap,totals_transactionRevenue_LogOfSum
0,0000000259678714014,7,0,2017-11-28,2017-11-28,2,1.5,1.5,1.5,1,2,21,12,1,2,5,18,129,94,26,119,1349,1.0,1.000000,1.000000,1,1,1.0,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0,0.0,0.000000,0.000000,...,48,48.0,48.0,48.0,48,48,332,332.0,332.000000,332.000000,332,332,4,4.0,4.0,4.0,4,4,0,1,23,11.5,11.5,11.5,0,23,26.0,26.0,26.0,19,33,30.5,30.5,30.5,21,40,2,1,0,0.0
1,0000049363351866189,5,1,2017-09-18,2017-09-19,3,2.0,2.0,2.0,1,3,21,12,1,2,5,18,130,94,26,119,1349,4.0,12.333333,12.333333,2,31,4.0,9.333333,9.333333,2.0,22.0,1.0,21.333333,21.333333,1.0,62.0,194.0,420.666667,420.666667,...,38,38.0,38.0,38.0,38,38,261,261.0,261.333333,261.333333,261,262,3,3.0,3.0,3.0,3,3,1,1,23,4.0,9.0,9.0,0,23,31.0,27.0,27.0,3,47,14.0,18.0,18.0,6,34,1,1,0,0.0


In [ ]:
split_3 = apply_to_splits(split_3, split_3_pre_gap_start, split_3_pre_gap_end)

In [ ]:
split_3.head(2)

,fullVisitorId,channelGrouping_ModeMax,date_Span,visitNumber_ModeMax,visitNumber_Median,visitNumber_Mean,visitNumber_Sum,visitNumber_Min,visitNumber_Max,device_browser_ModeMax,device_operatingSystem_ModeMax,device_isMobile_ModeMax,device_deviceCategory_ModeMax,geoNetwork_continent_ModeMax,geoNetwork_subContinent_ModeMax,geoNetwork_country_ModeMax,geoNetwork_region_ModeMax,geoNetwork_metro_ModeMax,geoNetwork_city_ModeMax,geoNetwork_networkDomain_ModeMax,totals_hits_Median,totals_hits_Mean,totals_hits_Sum,totals_hits_Min,totals_hits_Max,totals_pageviews_Median,totals_pageviews_Mean,totals_pageviews_Sum,totals_pageviews_Min,totals_pageviews_Max,totals_sessionQualityDim_Median,totals_sessionQualityDim_Mean,totals_sessionQualityDim_Sum,totals_sessionQualityDim_Min,totals_sessionQualityDim_Max,totals_timeOnSite_Median,totals_timeOnSite_Mean,totals_timeOnSite_Sum,totals_timeOnSite_Min,totals_timeOnSite_Max,...,weekOfYear_Mean,weekOfYear_Sum,weekOfYear_Min,weekOfYear_Max,dayOfYear_ModeMax,dayOfYear_Median,dayOfYear_Mean,dayOfYear_Sum,dayOfYear_Min,dayOfYear_Max,quarter_ModeMax,quarter_Median,quarter_Mean,quarter_Sum,quarter_Min,quarter_Max,isYearEnd_ModeMax,is_weekend_ModeMax,visit_hr_ModeMax,visit_hr_Median,visit_hr_Mean,visit_hr_Sum,visit_hr_Min,visit_hr_Max,visit_min_Median,visit_min_Mean,visit_min_Sum,visit_min_Min,visit_min_Max,visit_sec_Median,visit_sec_Mean,visit_sec_Sum,visit_sec_Min,visit_sec_Max,time_of_day_ModeMax,is_office_hours_ModeMax,returnedPostGap,totals_transactionRevenue_LogOfSum,firstVisit_AfterStart,lastVisit_BeforeEnd
0,0000000259678714014,7,0,2,1.5,1.5,1.5,1,2,21,12,1,2,5,18,129,94,26,119,1349,1.0,1.000000,1.000000,1,1,1.0,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0,0.0,0.000000,0.000000,0.0,0.0,...,48.0,48.0,48,48,332,332.0,332.000000,332.000000,332,332,4,4.0,4.0,4.0,4,4,0,1,23,11.5,11.5,11.5,0,23,26.0,26.0,26.0,19,33,30.5,30.5,30.5,21,40,2,1,0,0.0,148,19
1,0000049363351866189,5,1,3,2.0,2.0,2.0,1,3,21,12,1,2,5,18,130,94,26,119,1349,4.0,12.333333,12.333333,2,31,4.0,9.333333,9.333333,2.0,22.0,1.0,21.333333,21.333333,1.0,62.0,194.0,420.666667,420.666667,3.0,1065.0,...,38.0,38.0,38,38,261,261.0,261.333333,261.333333,261,262,3,3.0,3.0,3.0,3,3,1,1,23,4.0,9.0,9.0,0,23,31.0,27.0,27.0,3,47,14.0,18.0,18.0,6,34,1,1,0,0.0,77,89


In [ ]:
split_3.to_csv("/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/split_3_visitor_level.csv",index=False)

#### <font color="#b00be5"><b>Split - 4</b></font>

In [ ]:
%%time
split_4_visitorLevel = vistor_level_data(split_4_pre_gap_df)

CPU times: user 1h 2min 7s, sys: 2min 24s, total: 1h 4min 32s
Wall time: 1h 1min 33s


In [ ]:
split_4_visitorLevel.shape

(366202, 116)

In [ ]:
# dropping the multi-index level, resetting index and set fullVisitorId as the 1st column.
split_4_visitorLevel.columns = split_4_visitorLevel.columns.droplevel()
split_4_visitorLevel = split_4_visitorLevel.reset_index()

In [ ]:
split_4_visitorLevel.shape

(366202, 117)

In [ ]:
# merge revenue and returned data with other features.
split_4 = pd.merge(left=split_4_visitorLevel,right=split_4_totalRevenue_and_detectReturn_df,how="left",on="fullVisitorId")
split_4.shape

(366202, 119)

In [ ]:
split_4.head(2)

,fullVisitorId,channelGrouping_ModeMax,date_Span,date_FirstVisit,date_LastVisit,visitNumber_ModeMax,visitNumber_Median,visitNumber_Mean,visitNumber_Sum,visitNumber_Min,visitNumber_Max,device_browser_ModeMax,device_operatingSystem_ModeMax,device_isMobile_ModeMax,device_deviceCategory_ModeMax,geoNetwork_continent_ModeMax,geoNetwork_subContinent_ModeMax,geoNetwork_country_ModeMax,geoNetwork_region_ModeMax,geoNetwork_metro_ModeMax,geoNetwork_city_ModeMax,geoNetwork_networkDomain_ModeMax,totals_hits_Median,totals_hits_Mean,totals_hits_Sum,totals_hits_Min,totals_hits_Max,totals_pageviews_Median,totals_pageviews_Mean,totals_pageviews_Sum,totals_pageviews_Min,totals_pageviews_Max,totals_sessionQualityDim_Median,totals_sessionQualityDim_Mean,totals_sessionQualityDim_Sum,totals_sessionQualityDim_Min,totals_sessionQualityDim_Max,totals_timeOnSite_Median,totals_timeOnSite_Mean,totals_timeOnSite_Sum,...,weekOfYear_ModeMax,weekOfYear_Median,weekOfYear_Mean,weekOfYear_Sum,weekOfYear_Min,weekOfYear_Max,dayOfYear_ModeMax,dayOfYear_Median,dayOfYear_Mean,dayOfYear_Sum,dayOfYear_Min,dayOfYear_Max,quarter_ModeMax,quarter_Median,quarter_Mean,quarter_Sum,quarter_Min,quarter_Max,isYearEnd_ModeMax,is_weekend_ModeMax,visit_hr_ModeMax,visit_hr_Median,visit_hr_Mean,visit_hr_Sum,visit_hr_Min,visit_hr_Max,visit_min_Median,visit_min_Mean,visit_min_Sum,visit_min_Min,visit_min_Max,visit_sec_Median,visit_sec_Mean,visit_sec_Sum,visit_sec_Min,visit_sec_Max,time_of_day_ModeMax,is_office_hours_ModeMax,returnedPostGap,totals_transactionRevenue_LogOfSum
0,0000053049821714864,2,0,2018-01-28,2018-01-28,1,1.0,1.0,1.0,1,1,21,13,1,2,4,11,49,94,26,119,1349,6.0,6.0,6.0,6,6,5.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,70.0,70.0,70.0,...,4,4.0,4.000000,4.000000,4,4,28,28.0,28.0,28.0,28,28,1,1.0,1.0,1.0,1,1,1,0,22,22.0,22.000000,22.000000,22,22,32.0,32.000000,32.000000,32,32,22.0,22.000000,22.000000,22,22,2,1,0,0.0
1,0000059488412965267,5,6,2018-02-21,2018-02-27,3,2.0,2.0,2.0,1,3,21,13,1,2,5,18,130,93,26,112,1349,1.0,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,9,9.0,8.666667,8.666667,8,9,58,58.0,56.0,56.0,52,58,1,1.0,1.0,1.0,1,1,1,1,23,8.0,11.333333,11.333333,3,23,8.0,9.333333,9.333333,6,14,17.0,29.666667,29.666667,13,59,2,1,0,0.0


In [ ]:
split_4 = apply_to_splits(split_4, split_4_pre_gap_start, split_4_pre_gap_end)

In [ ]:
split_4.head(2)

,fullVisitorId,channelGrouping_ModeMax,date_Span,visitNumber_ModeMax,visitNumber_Median,visitNumber_Mean,visitNumber_Sum,visitNumber_Min,visitNumber_Max,device_browser_ModeMax,device_operatingSystem_ModeMax,device_isMobile_ModeMax,device_deviceCategory_ModeMax,geoNetwork_continent_ModeMax,geoNetwork_subContinent_ModeMax,geoNetwork_country_ModeMax,geoNetwork_region_ModeMax,geoNetwork_metro_ModeMax,geoNetwork_city_ModeMax,geoNetwork_networkDomain_ModeMax,totals_hits_Median,totals_hits_Mean,totals_hits_Sum,totals_hits_Min,totals_hits_Max,totals_pageviews_Median,totals_pageviews_Mean,totals_pageviews_Sum,totals_pageviews_Min,totals_pageviews_Max,totals_sessionQualityDim_Median,totals_sessionQualityDim_Mean,totals_sessionQualityDim_Sum,totals_sessionQualityDim_Min,totals_sessionQualityDim_Max,totals_timeOnSite_Median,totals_timeOnSite_Mean,totals_timeOnSite_Sum,totals_timeOnSite_Min,totals_timeOnSite_Max,...,weekOfYear_Mean,weekOfYear_Sum,weekOfYear_Min,weekOfYear_Max,dayOfYear_ModeMax,dayOfYear_Median,dayOfYear_Mean,dayOfYear_Sum,dayOfYear_Min,dayOfYear_Max,quarter_ModeMax,quarter_Median,quarter_Mean,quarter_Sum,quarter_Min,quarter_Max,isYearEnd_ModeMax,is_weekend_ModeMax,visit_hr_ModeMax,visit_hr_Median,visit_hr_Mean,visit_hr_Sum,visit_hr_Min,visit_hr_Max,visit_min_Median,visit_min_Mean,visit_min_Sum,visit_min_Min,visit_min_Max,visit_sec_Median,visit_sec_Mean,visit_sec_Sum,visit_sec_Min,visit_sec_Max,time_of_day_ModeMax,is_office_hours_ModeMax,returnedPostGap,totals_transactionRevenue_LogOfSum,firstVisit_AfterStart,lastVisit_BeforeEnd
0,0000053049821714864,2,0,1,1.0,1.0,1.0,1,1,21,13,1,2,4,11,49,94,26,119,1349,6.0,6.0,6.0,6,6,5.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,70.0,70.0,70.0,70.0,70.0,...,4.000000,4.000000,4,4,28,28.0,28.0,28.0,28,28,1,1.0,1.0,1.0,1,1,1,0,22,22.0,22.000000,22.000000,22,22,32.0,32.000000,32.000000,32,32,22.0,22.000000,22.000000,22,22,2,1,0,0.0,41,126
1,0000059488412965267,5,6,3,2.0,2.0,2.0,1,3,21,13,1,2,5,18,130,93,26,112,1349,1.0,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,8.666667,8.666667,8,9,58,58.0,56.0,56.0,52,58,1,1.0,1.0,1.0,1,1,1,1,23,8.0,11.333333,11.333333,3,23,8.0,9.333333,9.333333,6,14,17.0,29.666667,29.666667,13,59,2,1,0,0.0,65,96


In [ ]:
split_4.to_csv("/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/split_4_visitor_level.csv",index=False)

#### <font color="#b00be5"><b>Concat to form Train</b></font>

In [ ]:
train_new = pd.concat([split_1,split_2,split_3,split_4],axis=0).reset_index(drop=True)
train_new.shape

(1417575, 119)

In [ ]:
train_new.head(2)

,fullVisitorId,channelGrouping_ModeMax,date_Span,visitNumber_ModeMax,visitNumber_Median,visitNumber_Mean,visitNumber_Sum,visitNumber_Min,visitNumber_Max,device_browser_ModeMax,device_operatingSystem_ModeMax,device_isMobile_ModeMax,device_deviceCategory_ModeMax,geoNetwork_continent_ModeMax,geoNetwork_subContinent_ModeMax,geoNetwork_country_ModeMax,geoNetwork_region_ModeMax,geoNetwork_metro_ModeMax,geoNetwork_city_ModeMax,geoNetwork_networkDomain_ModeMax,totals_hits_Median,totals_hits_Mean,totals_hits_Sum,totals_hits_Min,totals_hits_Max,totals_pageviews_Median,totals_pageviews_Mean,totals_pageviews_Sum,totals_pageviews_Min,totals_pageviews_Max,totals_sessionQualityDim_Median,totals_sessionQualityDim_Mean,totals_sessionQualityDim_Sum,totals_sessionQualityDim_Min,totals_sessionQualityDim_Max,totals_timeOnSite_Median,totals_timeOnSite_Mean,totals_timeOnSite_Sum,totals_timeOnSite_Min,totals_timeOnSite_Max,...,weekOfYear_Mean,weekOfYear_Sum,weekOfYear_Min,weekOfYear_Max,dayOfYear_ModeMax,dayOfYear_Median,dayOfYear_Mean,dayOfYear_Sum,dayOfYear_Min,dayOfYear_Max,quarter_ModeMax,quarter_Median,quarter_Mean,quarter_Sum,quarter_Min,quarter_Max,isYearEnd_ModeMax,is_weekend_ModeMax,visit_hr_ModeMax,visit_hr_Median,visit_hr_Mean,visit_hr_Sum,visit_hr_Min,visit_hr_Max,visit_min_Median,visit_min_Mean,visit_min_Sum,visit_min_Min,visit_min_Max,visit_sec_Median,visit_sec_Mean,visit_sec_Sum,visit_sec_Min,visit_sec_Max,time_of_day_ModeMax,is_office_hours_ModeMax,returnedPostGap,totals_transactionRevenue_LogOfSum,firstVisit_AfterStart,lastVisit_BeforeEnd
0,0000010278554503158,7,0,1,1.0,1.0,1.0,1,1,21,13,1,2,3,0,15,94,26,119,406,3.0,3.0,3.0,3,3,3.0,3.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,61.0,61.0,61.0,61.0,61.0,...,42.0,42.0,42,42,294,294.0,294.0,294.0,294,294,4,4.0,4.0,4.0,4,4,0,1,5,5.0,5.0,5.0,5,5,57.0,57.0,57.0,57,57,46.0,46.0,46.0,46,46,1,1,0,0.0,80,87
1,0000020424342248747,7,0,1,1.0,1.0,1.0,1,1,20,10,0,1,4,17,47,61,26,74,1349,4.0,4.0,4.0,4,4,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,51.0,51.0,51.0,51.0,51.0,...,48.0,48.0,48,48,335,335.0,335.0,335.0,335,335,4,4.0,4.0,4.0,4,4,0,1,7,7.0,7.0,7.0,7,7,55.0,55.0,55.0,55,55,1.0,1.0,1.0,1,1,0,1,0,0.0,121,46


In [ ]:
train_new.to_csv("/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/train_new_visitor_level.csv",index=False)

#### <font color="#b00be5"><b>Test</b></font>

In [ ]:
%%time
test_visitorLevel = vistor_level_data(test)

CPU times: user 51min 17s, sys: 2min, total: 53min 17s
Wall time: 50min 48s


In [ ]:
test_visitorLevel.shape

(296530, 117)

In [ ]:
# dropping the multi-index level, resetting index and set fullVisitorId as the 1st column.
test_visitorLevel.columns = test_visitorLevel.columns.droplevel()
test_visitorLevel = test_visitorLevel.reset_index()

In [ ]:
test_visitorLevel.shape

(296530, 118)

In [ ]:
# shifting the target column to the end (just for convenience):-
shift_column = test_visitorLevel.pop("totals_transactionRevenue_LogOfSum")
test_visitorLevel.insert(test_visitorLevel.shape[1],"totals_transactionRevenue_LogOfSum",shift_column)

In [ ]:
test_visitorLevel.head(2)

,fullVisitorId,channelGrouping_ModeMax,date_Span,date_FirstVisit,date_LastVisit,visitNumber_ModeMax,visitNumber_Median,visitNumber_Mean,visitNumber_Sum,visitNumber_Min,visitNumber_Max,device_browser_ModeMax,device_operatingSystem_ModeMax,device_isMobile_ModeMax,device_deviceCategory_ModeMax,geoNetwork_continent_ModeMax,geoNetwork_subContinent_ModeMax,geoNetwork_country_ModeMax,geoNetwork_region_ModeMax,geoNetwork_metro_ModeMax,geoNetwork_city_ModeMax,geoNetwork_networkDomain_ModeMax,totals_hits_Median,totals_hits_Mean,totals_hits_Sum,totals_hits_Min,totals_hits_Max,totals_pageviews_Median,totals_pageviews_Mean,totals_pageviews_Sum,totals_pageviews_Min,totals_pageviews_Max,totals_sessionQualityDim_Median,totals_sessionQualityDim_Mean,totals_sessionQualityDim_Sum,totals_sessionQualityDim_Min,totals_sessionQualityDim_Max,totals_timeOnSite_Median,totals_timeOnSite_Mean,totals_timeOnSite_Sum,...,dayOfWeek_Max,weekOfYear_ModeMax,weekOfYear_Median,weekOfYear_Mean,weekOfYear_Sum,weekOfYear_Min,weekOfYear_Max,dayOfYear_ModeMax,dayOfYear_Median,dayOfYear_Mean,dayOfYear_Sum,dayOfYear_Min,dayOfYear_Max,quarter_ModeMax,quarter_Median,quarter_Mean,quarter_Sum,quarter_Min,quarter_Max,isYearEnd_ModeMax,is_weekend_ModeMax,visit_hr_ModeMax,visit_hr_Median,visit_hr_Mean,visit_hr_Sum,visit_hr_Min,visit_hr_Max,visit_min_Median,visit_min_Mean,visit_min_Sum,visit_min_Min,visit_min_Max,visit_sec_Median,visit_sec_Mean,visit_sec_Sum,visit_sec_Min,visit_sec_Max,time_of_day_ModeMax,is_office_hours_ModeMax,totals_transactionRevenue_LogOfSum
0,0000018966949534117,7,0,2018-08-13,2018-08-13,1,1.0,1.0,1.0,1,1,21,8,0,1,4,3,109,94,26,119,1349,1.0,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1,0.0,0.0,0.0,...,0,33,33.0,33.0,33.0,33,33,225,225.0,225.0,225.0,225,225,3,3.0,3.0,3.0,3,3,1,1,3,3.0,3.0,3.0,3,3,17.0,17.0,17.0,17,17,14.0,14.0,14.0,14,14,1,1,0.0
1,0000039738481224681,5,0,2018-06-13,2018-06-13,1,1.0,1.0,1.0,1,1,21,8,0,1,5,18,130,93,25,118,1349,1.0,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1,0.0,0.0,0.0,...,2,24,24.0,24.0,24.0,24,24,164,164.0,164.0,164.0,164,164,2,2.0,2.0,2.0,2,2,1,1,1,1.0,1.0,1.0,1,1,16.0,16.0,16.0,16,16,14.0,14.0,14.0,14,14,1,1,0.0


In [ ]:
test_visitorLevel = apply_to_splits(test_visitorLevel, test["date"].min(), test["date"].max())

In [ ]:
# saving this list to re-use again while creating production pipeline
pre_gap_dates = [test["date"].min(), test["date"].max()]
f_path = "/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/pre_gap_dates.pkl"
with open(f_path,"wb") as f:
  pickle.dump(pre_gap_dates,f)
f.close()

In [ ]:
test_visitorLevel.head(2)

,fullVisitorId,channelGrouping_ModeMax,date_Span,visitNumber_ModeMax,visitNumber_Median,visitNumber_Mean,visitNumber_Sum,visitNumber_Min,visitNumber_Max,device_browser_ModeMax,device_operatingSystem_ModeMax,device_isMobile_ModeMax,device_deviceCategory_ModeMax,geoNetwork_continent_ModeMax,geoNetwork_subContinent_ModeMax,geoNetwork_country_ModeMax,geoNetwork_region_ModeMax,geoNetwork_metro_ModeMax,geoNetwork_city_ModeMax,geoNetwork_networkDomain_ModeMax,totals_hits_Median,totals_hits_Mean,totals_hits_Sum,totals_hits_Min,totals_hits_Max,totals_pageviews_Median,totals_pageviews_Mean,totals_pageviews_Sum,totals_pageviews_Min,totals_pageviews_Max,totals_sessionQualityDim_Median,totals_sessionQualityDim_Mean,totals_sessionQualityDim_Sum,totals_sessionQualityDim_Min,totals_sessionQualityDim_Max,totals_timeOnSite_Median,totals_timeOnSite_Mean,totals_timeOnSite_Sum,totals_timeOnSite_Min,totals_timeOnSite_Max,...,weekOfYear_Median,weekOfYear_Mean,weekOfYear_Sum,weekOfYear_Min,weekOfYear_Max,dayOfYear_ModeMax,dayOfYear_Median,dayOfYear_Mean,dayOfYear_Sum,dayOfYear_Min,dayOfYear_Max,quarter_ModeMax,quarter_Median,quarter_Mean,quarter_Sum,quarter_Min,quarter_Max,isYearEnd_ModeMax,is_weekend_ModeMax,visit_hr_ModeMax,visit_hr_Median,visit_hr_Mean,visit_hr_Sum,visit_hr_Min,visit_hr_Max,visit_min_Median,visit_min_Mean,visit_min_Sum,visit_min_Min,visit_min_Max,visit_sec_Median,visit_sec_Mean,visit_sec_Sum,visit_sec_Min,visit_sec_Max,time_of_day_ModeMax,is_office_hours_ModeMax,totals_transactionRevenue_LogOfSum,firstVisit_AfterStart,lastVisit_BeforeEnd
0,0000018966949534117,7,0,1,1.0,1.0,1.0,1,1,21,8,0,1,4,3,109,94,26,119,1349,1.0,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1,0.0,0.0,0.0,0.0,0.0,...,33.0,33.0,33.0,33,33,225,225.0,225.0,225.0,225,225,3,3.0,3.0,3.0,3,3,1,1,3,3.0,3.0,3.0,3,3,17.0,17.0,17.0,17,17,14.0,14.0,14.0,14,14,1,1,0.0,104,63
1,0000039738481224681,5,0,1,1.0,1.0,1.0,1,1,21,8,0,1,5,18,130,93,25,118,1349,1.0,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1,0.0,0.0,0.0,0.0,0.0,...,24.0,24.0,24.0,24,24,164,164.0,164.0,164.0,164,164,2,2.0,2.0,2.0,2,2,1,1,1,1.0,1.0,1.0,1,1,16.0,16.0,16.0,16,16,14.0,14.0,14.0,14,14,1,1,0.0,43,124


In [ ]:
test_visitorLevel.to_csv("/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/test_new_visitor_level.csv",index=False)

<font color="#8fce00"><h4><u><b>Note:</b></u></h4></font> 
We don't have any "returnedPostGap" in the new (visitor-level) test dataset. That's because, this new dataset is our key to predict the future revenues for all the <font color="#b00be5"><b>296530</b></font> visitors. Our model's will predict which customers among these will return and who won't. And if somebody comes returns in <font color="#b00be5"><b>1st Dec,2018 to 31st Jan,2019</b></font> how much did they spend.